In [20]:
import pandas as pd
from nltk.tokenize import word_tokenize

class preprocessing:

    # tokenize_sentence: tokenizes a sentence
    def tokenize_sentence(self, sentence):
        words = word_tokenize(sentence)
        return words

    # tokenize_column: tokenizes a column of a dataframe and saves it as column_tokens
    def tokenize_column(self, dataframe, column):
        tokens = list()
        # tokenize each element of the column
        for index, row in dataframe.iterrows():
            sentence_tokens = self.tokenize_sentence(dataframe.loc[index,column])
            tokens.append(sentence_tokens)
        # save tokens as a new column
        dataframe[f"{column}_tokens"] = pd.Series(tokens,index=dataframe.index)

In [21]:
preprocessing2 = preprocessing()
preprocessing2.tokenize_sentence("alex zam papapapap")

['alex', 'zam', 'papapapap']

In [22]:
test_set  = pd.read_csv("./data_sets/test_set.tsv", delimiter="\t",names=["id","title","content"],           header=0)
display(test_set.head(5))

,id,title,content
0,385,Tate & Lyle boss bags top award\n,\n Tate & Lyle's chief executive has been nam...
1,1984,Halo 2 sells five million copies\n,\n Microsoft is celebrating bumper sales of i...
2,986,MSPs hear renewed climate warning\n,\n Climate change could be completely out of ...
3,1387,Pavey focuses on indoor success\n,\n Jo Pavey will miss January's View From Gre...
4,1295,Tories reject rethink on axed MP\n,\n Sacked MP Howard Flight's local Conservati...


In [23]:
preprocessing2.tokenize_column(test_set,"content")
display(test_set.head(5))

,id,title,content,content_tokens
0,385,Tate & Lyle boss bags top award\n,\n Tate & Lyle's chief executive has been nam...,"[Tate, &, Lyle, 's, chief, executive, has, bee..."
1,1984,Halo 2 sells five million copies\n,\n Microsoft is celebrating bumper sales of i...,"[Microsoft, is, celebrating, bumper, sales, of..."
2,986,MSPs hear renewed climate warning\n,\n Climate change could be completely out of ...,"[Climate, change, could, be, completely, out, ..."
3,1387,Pavey focuses on indoor success\n,\n Jo Pavey will miss January's View From Gre...,"[Jo, Pavey, will, miss, January, 's, View, Fro..."
4,1295,Tories reject rethink on axed MP\n,\n Sacked MP Howard Flight's local Conservati...,"[Sacked, MP, Howard, Flight, 's, local, Conser..."
